Here we demonstrate the generating of polyhedrons, whos feasible region is either empty or not. 
When given an appropriate cost vector the area contains a feasible solution. 

Additionally, we are keeping our decision variables (n) and our constraints (m) constant, since the problem of varying input dimension to our model significnatly increases the complexlixty of the project. This topic will be explored in more detail later in the further research section. This part of the project will likely be cut out and made into its own repository.

We plot the polyhedron (for n = 2) and label whether it is bounded or not. 
With the cost vector, if the feasible solution is unbounded we can simply take the dual of it and have an infeasible problem.
We will be graphing the primal problem as demonstration.

The linear program will take the standard form:
minimize: z = c'x
subject to: Ax <= b


In [49]:
#NOTE: Here We Detail The Generation of Random Linear Programs
#! More Work May Need To Be Done Regarding The Making Of Truly Balanced Data Through Dimensionality and Attribute Varation

import pulp as pl
import numpy as np

def generate_random_linear_program(min=np.random.choice([0,1]), decision_variables=np.random.randint(-10,10), constraints=np.random.randint(-10,10), iteration=0):
    '''
    Creating trivally varible random linear programs.

    TODO:
    >Expand doc string
    >Expand actual todo
    '''
    #? Randomly Determines Whether To Generate A Maximize or Minimize Problem
    if min == 1:
        linear_program = pl.LpProblem(f"Linear_Program_{iteration}", pl.LpMinimize)
    else:
        linear_program = pl.LpProblem(f"Linear_Program_{iteration}", pl.LpMaximize)

    #? Generates a Dictionary of n Decision Variables Then Randomizes Upper or Lower Bound From Zero
    decision_variables = pl.LpVariable.dicts("x", range(decision_variables))
    for variable in decision_variables:
        lower = np.random.choice([0,1])
        if lower == 1:
            decision_variables[variable].lowBound = 0
        else:
            decision_variables[variable].upBound = 0

    #? For Each Decision Varible Create a Random Constant Such That: 
    #? Objective Function = (Variable 1, Random Constant 1) + (Variable 2, Random Constant 2) + ... + (Variable i, Random Constant i)
    objective_function = pl.LpAffineExpression([(decision_variables[i], np.random.randint(-10,10)) for i in range(len(decision_variables))])

    #? Adds Objective Function To The Linear Program
    linear_program += objective_function

    #? Generates m Constraints With Random Constants Similar To Objective Function
    for _ in range(constraints):
        temporary_function = pl.LpAffineExpression([(decision_variables[i], np.random.randint(-10,10)) for i in range(len(decision_variables))])
        equality_sign = np.random.choice([-1,1]) #? Constraint Such That >= or <= Random Constant
        equality_value = np.random.randint(-10,10) #? Random Constraint Value
        temporary_constraint = pl.LpConstraint(temporary_function, equality_sign, rhs=equality_value)
        linear_program += temporary_constraint

    return linear_program

example_linear_program = generate_random_linear_program(decision_variables=2, constraints=2)
print(example_linear_program)

Linear_Program_0:
MINIMIZE
-4*x_0 + -7*x_1 + 0
SUBJECT TO
_C1: 0 x_0 + 9 x_1 >= 0

_C2: 0 x_0 - 4 x_1 >= -3

VARIABLES
-inf <= x_0 <= 0 Continuous
-inf <= x_1 <= 0 Continuous



In [61]:
#NOTE: Here We Detial The Creation of Balanced Training Data for the Machine Learning Model
#! More Work May Need To Be Done Regarding The Making Of Truly Balanced Data By Eliminating The Maximum Number Of Biasing Attributes

import json
'''
#? Custom Encoder Used For NumPy Variables In Linear Program
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)
'''
'''
def pulp_extract(linear_program):

    return a_matrix, b_vector, c_vector, b_sign, v_sign
'''

def create_balanced_data(size = 1000):
    '''
    Creating trivially balanced training data.

    TODO:
    >Expand doc string
    >Expand actual todo
    '''
    n, m = 2, 2
    training_data = []
    last_category = None
    while len(training_data) < size:

        #? Creating Linear Program
        temporary_lp = generate_random_linear_program(decision_variables=n, constraints=m)
        solver = pl.PULP_CBC_CMD()

        #* Should Check Here To Eliminate Possible Biasing Attributes

        #? Formatting Linear Program
        #lp_formatted = pulp_extract(temporary_lp)
        objective = -1 #? 0 == Minimze, 1 == Maximize
        a_matrix = []
        b_vector = []
        c_vector = []
        b_sign = []
        v_sign = []


        #? Pre-Formatting Linear Program For Machine Learning Before Solving
        '''
        temporary_lp.to_json('preprocess_lp.json', cls=NpEncoder)
        lp_data = open('preprocess_lp.json')
        processed_lp = json.load(lp_data)
        lp_data.close()
        '''

        #? Solving Linear Program For Data Label Creation
        result = temporary_lp.solve(solver)
        status = pl.LpStatus[temporary_lp.status]

        #? Pairing Data and Label While Balancing Into Training Set
        if status == "Optimal" and (last_category == None or last_category == 1):
            lp_category = 0
            training_data.append([lp_formatted, lp_category])
            last_category = 0

        elif status == "Infeasible" and (last_category == None or last_category == 0):
            lp_category = 1
            training_data.append([lp_formatted, lp_category])
            last_category = 1

    return training_data

example_training_data = create_balanced_data(10)
print(f"Example Data Length: {len(example_training_data)}")

category_count = [0,0]
for item in example_training_data:
    if item[1] == 0:
        category_count[0] += 1
    else:
        category_count[1] += 1
print(f"Example Data Balance [Feasible, Infeasible]: {category_count}")
print(f"Example Entry: {example_training_data[0][0]}")

Example Data Length: 10
Example Data Balance [Feasible, Infeasible]: [5, 5]
Example Entry: {'objective': {'name': None, 'coefficients': [{'name': 'x_0', 'value': -10}, {'name': 'x_1', 'value': 7}]}, 'constraints': [{'sense': -1, 'pi': None, 'constant': 10, 'name': None, 'coefficients': [{'name': 'x_0', 'value': 9}, {'name': 'x_1', 'value': -3}]}, {'sense': -1, 'pi': None, 'constant': -3, 'name': None, 'coefficients': [{'name': 'x_0', 'value': -6}, {'name': 'x_1', 'value': 0}]}], 'variables': [{'lowBound': None, 'upBound': 0, 'cat': 'Continuous', 'varValue': None, 'dj': None, 'name': 'x_0'}, {'lowBound': 0, 'upBound': None, 'cat': 'Continuous', 'varValue': None, 'dj': None, 'name': 'x_1'}], 'parameters': {'name': 'Linear_Program_0', 'sense': 1, 'status': 0, 'sol_status': 0}, 'sos1': [], 'sos2': []}


In [51]:
#NOTE: Here We Generate The Training and Testing Data

import random
import pickle

training_data = create_balanced_data(1000)
print(f"Training Data Length: {len(training_data)}")
category_count = [0,0]
for item in training_data:
    if item[1] == 0:
        category_count[0] += 1
    else:
        category_count[1] += 1
print(f"Training Data Balance [Feasible, Infeasible]: {category_count}")
random.shuffle(training_data)

testing_data = create_balanced_data(1000)
print(f"Testing Data Length: {len(testing_data)}")
category_count = [0,0]
for item in testing_data:
    if item[1] == 0:
        category_count[0] += 1
    else:
        category_count[1] += 1
print(f"Testing Data Balance [Feasible, Infeasible]: {category_count}")
random.shuffle(testing_data)


Training Data Length: 1000
Training Data Balance [Feasible, Infeasible]: [500, 500]
Testing Data Length: 1000
Testing Data Balance [Feasible, Infeasible]: [500, 500]


In [58]:
#NOTE: Here We Preprocess The Training Data

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import TextVectorization

import pickle

x_train = pickle.load(open("x_train.pickle", "rb"))
x_test = pickle.load(open("x_test.pickle", "rb"))

y_train = pickle.load(open("y_train.pickle", "rb"))
y_train = np.array(y_train)
y_test = pickle.load(open("y_test.pickle", "rb"))
y_test = np.array(y_train)

vectorizer = TextVectorization(output_mode="int")
x_train = vectorizer.adapt(x_train)
x_train = np.array(x_train)

x_test = vectorizer.adapt(x_test)
x_test = np.array(x_test)

#x_train = tf.keras.utils.normalize(x_train, axis=1)  # scales data between 0 and 1
#x_test = tf.keras.utils.normalize(x_test, axis=1)  # scales data between 0 and 1

model = tf.keras.models.Sequential()  # a basic feed-forward model
model.add(tf.keras.layers.Flatten())  # takes our 28x28 and makes it 1x784
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))  # a simple fully-connected layer, 128 units, relu activation
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))  # a simple fully-connected layer, 128 units, relu activation
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))  # our output layer. 10 units for 10 classes. Softmax for probability distribution

model.compile(optimizer='adam',  # Good default optimizer to start with
              loss='sparse_categorical_crossentropy',  # how will we calculate our "error." Neural network aims to minimize loss.
              metrics=['accuracy'])  # what to track

model.fit(x_train, y_train, epochs=3)  # train the model

val_loss, val_acc = model.evaluate(x_test, y_test)  # evaluate the out of sample data with model
print(val_loss)  # model's loss (error)
print(val_acc)  # model's accuracy

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type NoneType).

In [ ]:
#NOTE Messing Around

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

mnist = tf.keras.datasets.mnist  # mnist is a dataset of 28x28 images of handwritten digits and their labels
(x_train, y_train),(x_test, y_test) = mnist.load_data()  # unpacks images to x_train/x_test and labels to y_train/y_test

x_train = tf.keras.utils.normalize(x_train, axis=1)  # scales data between 0 and 1
x_test = tf.keras.utils.normalize(x_test, axis=1)  # scales data between 0 and 1

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=3)  # train the model

val_loss, val_acc = model.evaluate(x_test, y_test)  # evaluate the out of sample data with model
print(val_loss)  # model's loss (error)
print(val_acc)  # model's accuracy

In [ ]:
model.save('epic_num_reader.model')

In [ ]:
new_model = tf.keras.models.load_model('epic_num_reader.model')
predictions = new_model.predict(x_test)
print(np.argmax(predictions[0]))
plt.imshow(x_test[0],cmap=plt.cm.binary)
plt.show()

In [ ]:
#NOTE: Here Is Just Messing Around and Graphing Of Linear Programs
import matplotlib.pyplot as plt

import numpy as np
from scipy.spatial import HalfspaceIntersection, ConvexHull
from scipy.optimize import linprog

import pulp

def feasible_point(A, b):
    # finds the center of the largest sphere fitting in the convex hull
    norm_vector = np.linalg.norm(A, axis=1)
    A_ = np.hstack((A, norm_vector[:, None]))
    b_ = b[:, None]
    c = np.zeros((A.shape[1] + 1,))
    c[-1] = -1
    res = linprog(c, A_ub=A_, b_ub=b[:, None], bounds=(None, None))
    return res.x[:-1]
    
def hs_intersection(A, b):
    interior_point = feasible_point(A, b)
    halfspaces = np.hstack((A, -b[:, None]))
    hs = HalfspaceIntersection(halfspaces, interior_point)
    return hs

def plt_halfspace(a, b, bbox, ax):
    if a[1] == 0:
        ax.axvline(b / a[0])
    else:
        x = np.linspace(bbox[0][0], bbox[0][1], 100)
        ax.plot(x, (b - a[0]*x) / a[1])

def add_bbox(A, b, xrange, yrange):
    A = np.vstack((A, [
        [-1,  0],
        [ 1,  0],
        [ 0, -1],
        [ 0,  1],
    ]))
    b = np.hstack((b, [-xrange[0], xrange[1], -yrange[0], yrange[1]]))
    return A, b

def solve_convex_set(A, b, bbox, ax=None):
    A_, b_ = add_bbox(A, b, *bbox)
    interior_point = feasible_point(A_, b_)
    hs = hs_intersection(A_, b_)
    points = hs.intersections
    hull = ConvexHull(points)
    return points[hull.vertices], interior_point, hs

def plot_convex_set(A, b, bbox, ax=None):
    # solve and plot just the convex set (no lines for the inequations)
    points, interior_point, hs = solve_convex_set(A, b, bbox, ax=ax)
    if ax is None:
        _, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.set_xlim(bbox[0])
    ax.set_ylim(bbox[1])
    ax.fill(points[:, 0], points[:, 1], 'r')
    return points, interior_point, hs

def plot_inequalities(A, b, c, bbox, ax=None):
    # solve and plot the convex set,
    # the inequation lines, and
    # the interior point that was used for the halfspace intersections
    points, interior_point, hs = plot_convex_set(A, b, bbox, ax=ax)
    #ax.plot(*interior_point, 'o')
    print(interior_point)
    for a_k, b_k in zip(A, b):
        plt_halfspace(a_k, b_k, bbox, ax)
    return points, interior_point, hs

def generate_linear_program(n, m, lower_entry_bound, upper_entry_bound):

    a_matrix = np.random.randint(lower_entry_bound,upper_entry_bound,size = (m,n))
    b_vector = np.random.randint(lower_entry_bound,upper_entry_bound,size = (m))
    c_vector = np.random.randint(lower_entry_bound,upper_entry_bound,size = (n))

    # Making Ax <= b with b being positive
    for i in range(len(b_vector)):
        if b_vector[i] <= 0:
            b_vector[i] = b_vector[i] * -1
            a_matrix[i] = a_matrix[i] * -1
    
    return (a_matrix, b_vector, c_vector)

n = 2
m = 3

lower_entry_bound = -10
upper_entry_bound = 10

a_matrix, b_vector, c_vector = generate_linear_program(n,m,lower_entry_bound,upper_entry_bound)

print(f'A \n{a_matrix}\n')
print(f'b \n{b_vector}\n')
print(f'c \n{c_vector}\n')

linear_program = pulp.LpProblem("Test Generation", pulp.LpMinimize)
solver = pulp.PULP_CBC_CMD()
#? Have to figure out a way to dynamically generate variables for LP or use a differnt method to check infeasible/unbounded
x1 = pulp.LpVariable('x1', cat='Continuous')
x2 = pulp.LpVariable('x2', cat='Continuous')
linear_program += c_vector[0] * x1 + c_vector[1] * x2, 'Z'
for i in range(len(a_matrix)):
    linear_program += a_matrix[i][0] * x1 + a_matrix[i][1] * x2 <= b_vector[i]
print(linear_program)
result = linear_program.solve(solver)
status = pulp.LpStatus[linear_program.status]
print(status)

plt.rcParams['figure.figsize'] = (10, 10)
bbox = [(-10, 10), (-10, 10)]
fig, ax = plt.subplots()
plot_inequalities(a_matrix, b_vector, c_vector, bbox, ax)

In [ ]:
#NOTE: Here is Just Testing Of The Random Linear Program Generation And JSON Outputs
import json

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

#? Testing Data Distribution
#! More Work May Need To Be Done Regarding Making Truly Balanced Data With No Biasing Variables
type_distribution = [0,0,0,0] #? Feasible, Unbounded, Infeasible, Error
n, m = 2, 2
for i in range(1):
    temporary_lp = generate_random_linear_program(decision_variables=n, constraints=m)
    temporary_lp.to_json("preproccessing_lp1.json", cls=NpEncoder)
    solver = pl.PULP_CBC_CMD()
    result = temporary_lp.solve(solver)
    status = pl.LpStatus[temporary_lp.status]
    temporary_lp.to_json("preproccessing_lp2.json", cls=NpEncoder)

    if status == "Optimal":
        type_distribution[0] += 1
    elif status == "Unbounded":
        type_distribution[1] += 1
    elif status == "Infeasible":
        type_distribution[2] += 1
    else:
        type_distribution[3] += 1

    data1 = open('preproccessing_lp1.json')
    meat1 = json.load(data1)
    data1.close()

    data2 = open('preproccessing_lp2.json')
    meat2 = json.load(data2)
    data2.close()

    print(temporary_lp)
    print(status)
    print(meat1)
    print(meat2)